In [1]:
import os
import time
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold

data_dir = '../data'

In [3]:
class DataSplitter():
    '''
    CLASS: DataSplitter = splits the data accordingly
    '''
    def __init__(self, X: pd.DataFrame, y: pd.Series, cv_mode: str, rand_mode: str, out_dir: str,
                 tsize: float, Kfolds: int):
        self.X = X
        self.y = y
        self.cv_mode = cv_mode
        self.rand_mode = rand_mode
        self.out_dir = out_dir
        
        self.tsize = tsize    # not used if cv_mode == 'kfold'
        self.Kfolds = Kfolds  # not used if cv_mode != 'kfold'
    
    def scale_X(self) -> None:
        sc = StandardScaler()
        self.X = sc.fit_transform(self.X)
    
    def _standard_split(self) -> None:
        '''
        METHOD: _standard_split = use train_test_split() to split the data via random seed or random_state=42
        '''
        if self.rand_mode == 'random':
            self.X_train, self.X_test, self.y_train, self.y_test = \
                train_test_split(self.X, self.y, test_size=self.tsize, stratify=self.y, random_state=math.floor(time.time()))
        else:
            self.X_train, self.X_test, self.y_train, self.y_test = \
                train_test_split(self.X, self.y, test_size=self.tsize, stratify=self.y, random_state=42)
    
    def _kfold_train_test_split(self, train: np.ndarray, test: np.ndarray) -> None:
        '''
        METHOD: _kfold_train_test_split = this acts as the K-fold'ed version of train_test_split()
        IN: train = a list of indices designating the train data (the rest of the folds)
            test = a list of indices designating the test data (a single fold)
        OUT: X_train, X_test, y_train, y_test = the actual data split by the folding
        '''
        # NOTE: StratifiedKFold() splits in terms of iloc indices
        X_train = self.X.iloc[train, :].values
        X_test = self.X.iloc[test, :].values
        y_train = self.y.iloc[train]
        y_test = self.y.iloc[test]
        return X_train, X_test, y_train, y_test
    
    def _kfold_split(self) -> None:
        '''
        METHOD: _kfold_split = this is the K-fold'ed version of _standard_split above
                               it uses StratifiedKFold() and _kfold_train_test_split() via random seed or otherwise
        '''
        if self.rand_mode == 'random':
            skf = StratifiedKFold(n_splits=self.Kfolds, shuffle=True, random_state=math.floor(time.time()))
        else:
            skf = StratifiedKFold(n_splits=self.Kfolds, shuffle=True, random_state=42)
        self.K_X_train = []
        self.K_X_test = []
        self.K_y_train = []
        self.K_y_test = []
        for train, test in skf.split(self.X, self.y):
            # print(train, test)
            X_train, X_test, y_train, y_test = self._kfold_train_test_split(train, test)
            self.K_X_train.append(X_train)
            self.K_X_test.append(X_test)
            self.K_y_train.append(y_train)
            self.K_y_test.append(y_test)
    
    def _book_keeping(self, y_train: pd.Series, y_test: pd.Series) -> None:
        '''
        METHOD: book_keeping = check the ratio of 1's to 0's in the splits
        '''
        y_train_count0 = len([sub for sub in y_train if sub == 0])
        y_train_count1 = len(y_train) - y_train_count0
        y_test_count0 = len([sub for sub in y_test if sub == 0])
        y_test_count1 = len(y_test) - y_test_count0
        
        print(f"  -- y_train has {y_train_count1} / {y_train_count0} = {y_train_count1/y_train_count0:.2f}x 1's to 0's")
        print(f"     y_test has {y_test_count1} / {y_test_count0} = {y_test_count1/y_test_count0:.2f}x 1's to 0's")
    
    def perform_splitting(self) -> None:
        '''
        METHOD: perform_splitting = run _kfold_split() or _standard_split()
        '''
        print("- some book keeping:")
        print(f"  compared to 564 / 192 = {564/192:.2f}x 1's (PD) to 0's (HC) in the full dataset,")
        if self.cv_mode == 'kfold':
            self._kfold_split()
            for kk in range(self.Kfolds):
                print(f"  < fold {kk} >")
                self._book_keeping(self.K_y_train[kk], self.K_y_test[kk])
        else:
            self._standard_split()
            self._book_keeping(self.y_train, self.y_test)
    
    def save(self):
        '''
        METHOD: save = save all the split out data into their respective csv's
        '''
        if not os.path.isdir(self.out_dir):
            os.makedirs(self.out_dir)
        
        print("- saving y's and X's to csv...")
        if self.cv_mode == 'kfold':
            for kk in range(self.Kfolds):
                print(f"  -- saving fold {kk}")
                self.K_y_train[kk].to_csv(self.out_dir + '/y_train-' + str(kk) + '_kfold' + str(self.Kfolds) + '.csv')
                self.K_y_test[kk].to_csv(self.out_dir + '/y_test-' + str(kk) + '_kfold' + str(self.Kfolds) + '.csv')
                np.savetxt(self.out_dir + '/X_train-' + str(kk) + '_kfold' + str(self.Kfolds) + '.csv', self.K_X_train[kk], delimiter=',')
                np.savetxt(self.out_dir + '/X_test-' + str(kk) + '_kfold' + str(self.Kfolds) + '.csv', self.K_X_test[kk], delimiter=',')
        else:
            self.y_train.to_csv(self.out_dir + '/y_train.csv')
            self.y_test.to_csv(self.out_dir + '/y_test.csv')
            np.savetxt(self.out_dir + '/X_train.csv', self.X_train, delimiter=',')
            np.savetxt(self.out_dir + '/X_test.csv', self.X_test, delimiter=',')
        print("  ...done.")
    
    def split_and_save(self) -> None:
        '''
        METHOD: split_and_save = run the above methods in succession
        '''
        self.perform_splitting()
        self.save()

In [24]:
df = pd.read_csv(data_dir + '/pd_speech_features_reduced.csv')
display(df)
print(df.shape)

y = df['class']
print(y)
features = list(df.columns)
features.remove('id')
features.remove('class')
X = df[features]

,id,DFA,numPulses,numPeriodsPulses,minIntensity,meanIntensity,f1,VFER_mean,VFER_entropy,mean_MFCC_2nd_coef,...,tqwt_maxValue_dec_11,tqwt_maxValue_dec_12,tqwt_maxValue_dec_13,tqwt_maxValue_dec_14,tqwt_maxValue_dec_15,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_36,class
0,0,0.71826,240,239,69.997496,72.465512,539.342735,0.000463,0.83621,2.48740,...,0.022796,0.024286,0.048924,0.105750,0.20486,1.6058,1.5466,1.5620,18.9405,1
1,0,0.69481,234,233,67.415903,71.528945,564.363615,0.000615,1.04550,2.89860,...,0.080975,0.099695,0.112210,0.121580,0.18155,1.5772,1.5530,1.5589,45.1780,1
2,0,0.67604,232,231,62.661706,68.086583,548.444604,0.000360,0.63668,3.22080,...,0.013810,0.026241,0.039886,0.081818,0.17028,1.5921,1.5399,1.5643,4.7666,1
3,1,0.79672,178,177,76.306989,79.190593,819.529588,0.000196,0.45452,3.10230,...,0.111290,0.122970,0.273530,0.567740,0.83417,1.8829,6.9761,3.7805,4.0603,1
4,1,0.79782,236,235,76.645686,79.183495,846.796144,0.000281,0.71541,2.94510,...,0.086621,0.102950,0.235860,0.567490,0.74186,1.8821,7.8832,6.1727,6.1164,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,250,0.56355,417,416,79.991789,81.244436,881.915768,0.019310,30.50930,-1.97830,...,0.300560,0.463150,0.646970,0.983470,0.98274,10.1906,3.4394,3.0706,6.8671,0
752,250,0.56499,415,413,70.191330,79.423824,907.787590,0.002685,6.58010,-1.45210,...,0.136800,0.270620,0.727660,1.012000,1.31670,22.4043,19.6733,1.9704,28.4170,0
753,251,0.72335,381,380,75.091300,76.485859,687.046383,0.000113,0.38927,-1.13730,...,0.168010,0.277380,0.348020,0.318670,0.30577,2.3279,137.3126,51.5607,5.0424,0
754,251,0.74890,340,339,76.168804,77.870915,771.948334,0.000138,0.40543,-0.17925,...,0.116660,0.211770,0.265250,0.266000,0.35645,1.6052,5.4425,19.1607,3.7131,0


(756, 165)
0      1
1      1
2      1
3      1
4      1
      ..
751    0
752    0
753    0
754    0
755    0
Name: class, Length: 756, dtype: int64


In [25]:
# split for: standard train_test_split

ds_standard = DataSplitter(X, y, 'standard', 'non-random', data_dir + '/split/standard', 0.2, -1)
ds_standard.split_and_save()

- some book keeping:
  compared to 564 / 192 = 2.94x 1's (PD) to 0's (HC) in the full dataset,
  -- y_train has 451 / 153 = 2.95x 1's to 0's
     y_test has 113 / 39 = 2.90x 1's to 0's
- saving y's and X's to csv...
  ...done.


In [26]:
# split for: stratified K-fold CV split

ds_kfold = DataSplitter(X, y, 'kfold', 'non-random', data_dir + '/split/kfold', -1, 5)
ds_kfold.split_and_save()

- some book keeping:
  compared to 564 / 192 = 2.94x 1's (PD) to 0's (HC) in the full dataset,
  < fold 0 >
  -- y_train has 451 / 153 = 2.95x 1's to 0's
     y_test has 113 / 39 = 2.90x 1's to 0's
  < fold 1 >
  -- y_train has 451 / 154 = 2.93x 1's to 0's
     y_test has 113 / 38 = 2.97x 1's to 0's
  < fold 2 >
  -- y_train has 451 / 154 = 2.93x 1's to 0's
     y_test has 113 / 38 = 2.97x 1's to 0's
  < fold 3 >
  -- y_train has 451 / 154 = 2.93x 1's to 0's
     y_test has 113 / 38 = 2.97x 1's to 0's
  < fold 4 >
  -- y_train has 452 / 153 = 2.95x 1's to 0's
     y_test has 112 / 39 = 2.87x 1's to 0's
- saving y's and X's to csv...
  -- saving fold 0
  -- saving fold 1
  -- saving fold 2
  -- saving fold 3
  -- saving fold 4
  ...done.


In [ ]:
# F- I-- N---